# Sentence Level Classification with BERT

Our goal is to train a classifier that can predict the CEFR level of any given sentence. In this notebook we will use 🤗[Hugging Face](https://huggingface.co/) and its transformers library as the training framework, with [Pytorch](https://pytorch.org/) as the deep learning backend.

There is a dataset containing sentences with the corresponding CEFR level, and we have to use BERT and train a sentence classifier with this dataset.

## Prepare your environment

As always, we install all packages with a virtual environment manager, like [venv](https://packaging.python.org/en/latest/guides/installing-using-pip-and-virtual-environments/) or [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html), to prevent version conflicts of different packages.  

### Install CUDA
Deep learning is a computionally extensive process. It takes lots of time if relying only on the CPU, especially when it's trained on a large dataset. That's why using GPU instead is generally recommended.  
To use GPU for computation, you have to install [CUDA toolkit](https://developer.nvidia.com/cuda-toolkit) as well as the [cuDNN library](https://developer.nvidia.com/cudnn) provided by NVIDIA.  


### Install python packages
The following python packages will be used in this tutorial:

1. `numpy`: for matrix operation
2. `scikit-learn`: for label encoding
3. `datasets`: for data preparation
4. `transformers`: for model loading and finetuing
5. `pytorch`: the backend DL framework
  - Note that the pt version must support the CUDA version you've installed if you want to use GPU.

### Select GPU(s) for your backend

Skip this section if you have no intension of using GPU with tensorflow/pytorch.

In [1]:
import os

# select your GPU. Note that this should be set before you load tensorflow or pytorch.
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['WANDB_DISABLE'] = 'True'

# To use multiple GPUs, combine all GPU ID with commas
# e.g. >>> os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,3'

In [2]:
import torch
print(torch.__version__)
# Check if any GPU is used
torch.cuda.is_available()

2.0.1+cu118


True

## Prepare the dataset

Before starting the training, we need to load and process our dataset - but wait, let's decide which model we want to use first.  

In the highly unlikely chance you've never heard of it, [BERT](https://arxiv.org/abs/1810.04805) (**B**idirectional **E**ncoder **R**epresentations from **T**ransformers) is a language model proposed by Google AI in 2018, and it's currently one of the most popular models used in NLP.  
You can learn more about it here:
- [BERT Explained: A Complete Guide with Theory and Tutorial](https://towardsml.com/2019/09/17/bert-explained-a-complete-guide-with-theory-and-tutorial/) by Samia, 2019.


However, we will not directly use BERT in this tutorial, because it's large and takes too long to train. Instead, we'll be using [DistilBert](https://medium.com/huggingface/distilbert-8cf3380435b5), a version of BERT that while light-weight, reserves 95% of its original accuracy.




In [3]:
# the model you want to use. Available models can be found here: https://huggingface.co/models
MODEL_NAME = 'distilbert-base-uncased'

### Load data

Similar to the `transformers` library, `datasets` is also a package by huggingface. It contains many public datasets online and can help us with the data processing.  
We can use `load_dataset` function to read the input `.csv` file provided for this assignment.

Reference:
 - [Official datasets document](https://huggingface.co/docs/datasets)
 - [datasets.load_dataset](https://huggingface.co/docs/datasets/loading.html)

In [4]:
# load the data using the load_dataset function
from datasets import load_dataset
dataset = load_dataset('./data')

dataset

Found cached dataset csv (/home/nlplab/roger/.cache/huggingface/datasets/csv/data-be30cc0ec9fc77bb/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'level'],
        num_rows: 20720
    })
    test: Dataset({
        features: ['text', 'level'],
        num_rows: 2300
    })
})

In [5]:
print(dataset['train'])
print(dataset['train'][1])
print(dataset['train']['text'][:5])

Dataset({
    features: ['text', 'level'],
    num_rows: 20720
})
{'text': 'You can contact me by e-mail.', 'level': 'A1'}
['My mother is having her car repaired.', 'You can contact me by e-mail.', 'He had a break for the weekend, and he called me: "I am in London, so, if you want to see me, it\'s the time!"', "Research shows that 40 percent of the program's viewers are aged over 55.", "I'd guess she's about my age."]


### Preprocessing

As always, texts should be tokenized, embedded, and padded before being put into the model.  
But not to worry, there are libraries from huggingface to help with this, too.

#### Sentence processing

Different pre-trained language models may have their own preprocessing models, and that's why we should use the tokenizers trained along with that model. In our case, we are using distilBERT, so we should use the distilBERT tokenizer.  

With huggingface, loading different tokenizers is extremely easy: just import the AutoTokenizer from `transformers` and tell it what model you plan to use, and it will handle everything for you.

Reference:
 - [transformers.AutoTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoTokenizer)

In [3]:
# load the distilBERT tokenizer using AutoTokenizer

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

#### Label processing

Our labels also need to be processed, so let's do that next.

For this tutorial, we'll use the OneHotEncoder provided by scikit-learn.

For now, just declare a new encoder and use `fit` to learn the data. Hint: you should still end up with 6 labels.

Documents:
 - [sklearn.preprocessing.OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder)

In [7]:
# declare a new encoder and let it learn from the dataset
from sklearn.preprocessing import OneHotEncoder
import numpy

encoder = OneHotEncoder()
encoder.fit(numpy.array(dataset['train']['level']).reshape(-1,1))

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [8]:
# check if you still have 6 labels
LABEL_COUNT = len(encoder.categories_[0])
print(LABEL_COUNT)

6


#### Process the data

To make things easier, we can write a function to process our dataset in batches. 

In [9]:
import numpy as np

def preprocess(dataslice): # dataset['train'][:10]
    """ Input: a batch of your dataset
        Example: { 'text': [['sentence1'], ['setence2'], ...],
                   'label': ['label1', 'label2', ...] }
    """
    
    # use your tokenizor and encoder to get sentence embeddings and encoded labels
    # print(dataslice['text'])
    
    dataslice['input_ids'] = []
    dataslice['attention_mask'] = []
    dataslice['label'] = []
    
    for idx in range(len(dataslice['text'])):
        text = dataslice['text'][idx]
        level = dataslice['level'][idx]
        
        t = tokenizer(text)
        l = encoder.transform([[level]]).toarray()
        
        dataslice['input_ids'].append(t['input_ids'])
        dataslice['attention_mask'].append(t['attention_mask'])
        dataslice['label'].append(np.reshape(l, (6)))
        
    """ Output: a batch of processed dataset
        Example: { 'input_ids': ...,
                   'attention_masks': ...,
                   'label': ... }
    """
    
    return dataslice


In [10]:
# map the function to the whole dataset
processed_data = dataset.map(preprocess,    # your processing function
                             batched = True # Process in batches so it can be faster
                            )

Loading cached processed dataset at /home/nlplab/roger/.cache/huggingface/datasets/csv/data-be30cc0ec9fc77bb/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-13c7fb7d4be74de6.arrow
Loading cached processed dataset at /home/nlplab/roger/.cache/huggingface/datasets/csv/data-be30cc0ec9fc77bb/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-2dde0fa24c7de972.arrow


In [11]:
print(processed_data)
processed_data['train'][0]

DatasetDict({
    train: Dataset({
        features: ['text', 'level', 'input_ids', 'attention_mask', 'label'],
        num_rows: 20720
    })
    test: Dataset({
        features: ['text', 'level', 'input_ids', 'attention_mask', 'label'],
        num_rows: 2300
    })
})


{'text': 'My mother is having her car repaired.',
 'level': 'B1',
 'input_ids': [101, 2026, 2388, 2003, 2383, 2014, 2482, 13671, 1012, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'label': [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]}

### DataCollator

You might have noticed that we skipped padding the sentences. That's because we are going to do it during training.  

To do training-time processing, we can use the DataCollator Class provided by `transformers`. And guess what - transformers has a class that will handle padding for us, too!

 - [transformers.DataCollatorWithPadding](https://huggingface.co/docs/transformers/master/en/main_classes/data_collator#transformers.DataCollatorWithPadding)

In [12]:
# declare a collator to do padding during traning
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer)

## Training

Finally, we can move on to training.

### Preparation

We can load the pretrained model from `transformers`.  
Generally, you need to build your own model on top of BERT if you want to use BERT for some downstream tasks, but again, sequence classification is a popular topic. With the support from `transformers` library, it can be done in two lines of codes: 

1. Load `AutoModelForSequenceClassification` Class.
2. Load the pretrained model.

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased',
                                                           num_labels = LABEL_COUNT)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

#### Split train/val data

The `Dataset` class we prepared before has a `train_test_split` method. You can use it to split your (processed) dataset.

Document:
 - [datasets.Dataset - Sort, shuffle, select, split, and shard](https://huggingface.co/docs/datasets/process.html#sort-shuffle-select-split-and-shard)

In [14]:
# choose a validation size and split your data
train_val_dataset = processed_data['train'].train_test_split(test_size=0.2, shuffle=True)

In [15]:
print(train_val_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'level', 'input_ids', 'attention_mask', 'label'],
        num_rows: 16576
    })
    test: Dataset({
        features: ['text', 'level', 'input_ids', 'attention_mask', 'label'],
        num_rows: 4144
    })
})


#### Setup training parameters

We are using the TrainerAPI to do the training. Trainer is yet another utility provided by huggingface, which helps you train the model with ease.  

Document:
- [transformers.TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments)
- [transformers.Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer)

In [16]:
from transformers import TrainingArguments, Trainer

In [17]:
# set and tune your training properties
from transformers import TrainingArguments, Trainer

OUTPUT_DIR = './'
LEARNING_RATE = 5e-5
BATCH_SIZE = 100
EPOCH = 5
training_args = TrainingArguments(
    output_dir = OUTPUT_DIR,
    learning_rate = LEARNING_RATE,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    num_train_epochs = EPOCH,
    report_to = 'none'
    # you can set more parameters here if you want
)

# now give all the information to a trainer
trainer = Trainer(
    # set your parameters here
    model = model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_val_dataset['train'],
    eval_dataset = train_val_dataset['test']
)

### Training

This is the easy part. Simply ask the trainer to train the model for you!

In [18]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.315800


TrainOutput(global_step=830, training_loss=0.25925249306552384, metrics={'train_runtime': 207.4767, 'train_samples_per_second': 399.467, 'train_steps_per_second': 4.0, 'total_flos': 1187596984846464.0, 'train_loss': 0.25925249306552384, 'epoch': 5.0})

### Save for future use

Hint: try using `save_pretrained`

In [19]:
# practice saving your model for future use
model.save_pretrained("./")

## Prediction

Now we know exactly how to train a model, but how do we use it for predicting results?

### Load finetuned model

In [20]:
# load the model that you saved
mymodel = AutoModelForSequenceClassification.from_pretrained('./')

### Get the prediction

Here are a few example sentences:

In [21]:
examples = [
    # A2
    "Remember to write me a letter.",
    # B2
    "Strawberries and cream - a perfect combination.",
    "This so-called \"Perfect Evening\" was so disappointing, as well as discouraging us from coming to your Circle Theatre again.",
    # C1
    "Some may altogether give up their studies, which I think is a disastrous move.",
]

All we need to do is to transform them to embeddings, and then we can get predictions by calling your finetuned model.  

Since we don't have a DataCollator to pad the sentence and do the matrix transformation this time, we have to pad and transform the matrice on our own.

In [22]:
# Transform the sentences into embeddings
input = tokenizer(examples, truncation=True, padding=True, return_tensors="pt")
# Get the output
logits = mymodel(**input).logits
logits

tensor([[-0.9237,  0.8402, -3.1462, -5.0538, -4.1780, -4.6199],
        [-6.6779, -6.2395, -3.7227, -0.0955, -1.5378, -2.6053],
        [-5.7536, -5.4781, -5.2673, -1.1857,  1.3211, -3.6174],
        [-5.7480, -5.6256, -3.6208,  2.3783, -2.5812, -3.8506]],
       grad_fn=<AddmmBackward0>)

Logits aren't very readable for us. Let's use softmax 
activation to transform them into more probability-like numbers.

In [23]:
from torch import nn

predicts = nn.functional.softmax(logits, dim = -1)
print(predicts)
#torch.max(predicts, 1).indices

tensor([[1.4239e-01, 8.3086e-01, 1.5425e-02, 2.2899e-03, 5.4976e-03, 3.5336e-03],
        [1.0273e-03, 1.5925e-03, 1.9730e-02, 7.4195e-01, 1.7539e-01, 6.0307e-02],
        [7.7487e-04, 1.0207e-03, 1.2602e-03, 7.4655e-02, 9.1573e-01, 6.5610e-03],
        [2.9215e-04, 3.3017e-04, 2.4515e-03, 9.8805e-01, 6.9328e-03, 1.9482e-03]],
       grad_fn=<SoftmaxBackward0>)


#### Transform logits back to labels(10pts)

Now you've got the output. Write a function to map it back into labels!

In [24]:
# try to process the result
def convertToLabels(logits):
    d = {
        0: 'A1',
        1: 'A2',
        2: 'B1',
        3: 'B2',
        4: 'C1',
        5: 'C2'
    }
    return list(map(d.get, list(torch.max(logits, 1).indices.detach().numpy())))

convertToLabels(predicts)

['A2', 'B2', 'C1', 'B2']

## Evaluation

Let's see how you did!  
Load the testing data and calculate your accuracy.

We want you to calculate the three kinds of accuracy mentioned in the lecture, which will also be explained in the following section.

In [25]:
# load test data
# preprocess
# get predictions
input_test = tokenizer(processed_data['test']['text'][:], truncation=True, padding=True, return_tensors="pt")

logits_test = mymodel(**input_test).logits
predicts_test = nn.functional.softmax(logits_test, dim = -1)
predict_label_test = convertToLabels(predicts_test)
# transform predictions back into labels

In [26]:
#  try printing out some predictions to check if the outputs are reasonable and if you need to adjust your model at the end of every step.

for idx, (sent, level) in enumerate(zip(processed_data['test']['text'], predict_label_test)):
    if idx >= 10: break
    print(f'{level}: {sent}') 

C2: No longer a remote, backward, unimportant country, it became a force to be reckoned with in Europe.
B2: Unfortunately he was too fast and I couldn't keep up with him.
B2: Most mushrooms are totally harmless, but some are poisonous.
C2: This provided solid evidence that he committed the crime.
C1: You can't just accept everything you read in the newspapers at face value.
A2: Remember to write me a letter.
B1: She has long blond hair and blue eyes. She has a good figure.
B2: Nowadays the aim in clothing is not just for covering and protecting ourselves.
A2: Take two tablets, three times a day.
C2: Well, you will be if you saw our slide show and talk - members can hardly forget that relaxing afternoon when we unfolded the sails on the lake and enjoyed the tranquility of the area.


### Six Level Accuracy(30pts)

Exact accuracy is probably what you're most familiar with:

$
accuracy = \frac{\#exactly\:the\:same\:levels}{\#total}
$

Example:
```
Prediction:   A1 A2 B1 B2 C1 C2
Ground truth: A2 B1 B1 B2 B2 C2
                    ^  ^     ^
```

The six level accuracy is $\frac{3}{6} = 0.5$

As the requirement, <u>your exact accuracy should be higher than $0.5$</u>.

In [27]:
# calculate accuracy
correct = 0
size = len(processed_data['test']['level'])
for idx in range(size):
    if (predict_label_test[idx] == processed_data['test']['level'][idx]):
        correct += 1

print(correct / size)

0.5230434782608696


### Three Level Accuracy(30pts)

Three Level Accuracy is used when you only want a more general sense of right or wrong.

$
accuracy = \frac{\#the\:same\:ABC\:levels}{\#total}
$

Example:
```
Prediction:   A1 A2 B1 B2 C1 C2
Ground truth: A2 B1 B1 B2 B2 C2
              ^     ^  ^     ^
```

The three level accuracy is $\frac{4}{6} = 0.667$

As the requirement, <u>your exact accuracy should be higher than $0.6$</u>.

In [28]:
# calculate accuracy
correct = 0
for idx in range(size):
    if (predict_label_test[idx][0] == processed_data['test']['level'][idx][0]):
        correct += 1

print(correct / size)

0.7217391304347827


### Fuzzy accuracy(30pts)

However, the level of a sentence is relatively subjective. Generally speaking, $\pm1$ errors are allowed in the real evaluation in linguistic area.  

For example, if the actual label is 'B1', we'll also consider the prediction 'right' if the model predicts 'B2' or 'A2'.

Hence, the fuzzy accuracy is

$
accuracy = \frac{\#good\:enough\:answers}{\#total}
$

Example:
```
Prediction:   0 1 2 3 4 5
Ground truth: 0 1 1 3 3 3
              ^ ^ ^ ^ ^
```

The fuzzy accuracy is $\frac{5}{6} = 0.833$

As the requirement, <u>your accuracy should be higher than $0.8$</u>.

In [29]:
# calculate accuracy
l = ['dummy', 'A1', 'A2', 'B1', 'B2', 'C1', 'C2', 'dummy']

correct = 0
for idx in range(size):
    arr_index = l.index(processed_data['test']['level'][idx])
    if (predict_label_test[idx] in l[arr_index-1:arr_index+2]):
        correct += 1

print(correct / size)

0.8630434782608696


## Appendix 


<a name="Appendix-1-Install-CUDA"></a>

### Appendix 1 - Install CUDA

1. Check your GPU vs. CUDA compatibility:
   - [NVIDIA -> Your GPU Compute Capability](https://developer.nvidia.com/cuda-gpus) -> GeForce and TITAN Products
2. Check library vs. CUDA compatibility: 
   - Pytorch: [Previous PyTorch Versions](https://pytorch.org/get-started/previous-versions/)
   - Tensorflow: [Linux/MacOX](https://www.tensorflow.org/install/source#tested_build_configurations) or [Windows](https://www.tensorflow.org/install/source_windows#tested_build_configurations)
3. Note the highest CUDA version that fits your system.

#### >> for conda/mamba users

You can directly install CUDA library with the selected CUDA version.
1. Get [the driver for NVIDIA GPU](https://www.nvidia.com/download/index.aspx)
2. `conda/mamba install -c conda-forge cudatoolkit=${VERSION}`

#### >> for non-conda users

1. Get [the driver for NVIDIA GPU](https://www.nvidia.com/download/index.aspx)
2. Download and install [CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit-archive)
3. Download and install [cuDNN Library](https://developer.nvidia.com/rdp/cudnn-archive)

### Appendix 2 - Further Readings

1. [Huggingface Official Tutorials](https://github.com/huggingface/notebooks/tree/master/examples)
2. How to use Bert with other downstream tasks: [How to use BERT from the Hugging Face transformer library](https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209): 
3. Training with pytorch backend: [transformers-tutorials](https://github.com/abhimishra91/transformers-tutorials)
4. A more complicated example that include manual data/training processing with Pytorch: [Transformers for Multi-Label Classification made simple](https://towardsdatascience.com/transformers-for-multilabel-classification-71a1a0daf5e1)
5. [Text Classification with tensorflow](https://github.com/huggingface/notebooks/blob/master/examples/text_classification-tf.ipynb): tensorflow example